# ARC 2018 Model #

In [3]:
import openmc
import neutronics_material_maker as nmm
import math


## Materials ##

All material management is done using `neutronics_material_maker`. The name of each material must correspond to the name of the region of the DAGMC model to which it should be assigned.

In [4]:
nmm.AddMaterialFromFile("mat_lib.json")

# Material Parameters:
Li6_enrichment = 7.8
U235_enrichment = 0.007
percent_fertile = 5 / 100

plasma = nmm.Material.from_library("plasma").openmc_material

flibe = nmm.Material.from_library("flibe", enrichment=Li6_enrichment)
uf4 = nmm.Material.from_library("uf4")

#Creating materials for all four FLiBe regions by mixing UF4 and FLiBe materials by volume
ib_tank = nmm.Material.from_mixture(name='inboard_tank', materials=[flibe, uf4], fracs=[1 - percent_fertile, percent_fertile], percent_type='vo').openmc_material
ob_tank = nmm.Material.from_mixture(name='outboard_tank', materials=[flibe, uf4], fracs=[1 - percent_fertile, percent_fertile], percent_type='vo').openmc_material

ib_cc = nmm.Material.from_mixture(name='inboard_cc', materials=[flibe, uf4], fracs=[1 - percent_fertile, percent_fertile], percent_type='vo').openmc_material
ob_cc = nmm.Material.from_mixture(name='outboard_cc', materials=[flibe, uf4], fracs=[1 - percent_fertile, percent_fertile], percent_type='vo').openmc_material

ib_mult = nmm.Material.from_library("lead").openmc_material
ib_mult.name = 'inboard_multiplier'

ob_mult = nmm.Material.from_library("lead").openmc_material
ob_mult.name = 'outboard_multiplier'

ob_outer_vv = nmm.Material.from_library("Inconel-718").openmc_material
ob_outer_vv.name = 'outboard_outer_vv'

ib_outer_vv = nmm.Material.from_library("Inconel-718").openmc_material
ib_outer_vv.name = 'inboard_outer_vv'

vv = nmm.Material.from_library("tungsten").openmc_material
vv.name = "vv"

materials = openmc.Materials(
    [
        plasma,
        ib_tank,
        ob_tank,
        ib_cc,
        ob_cc,
        ib_mult,
        ob_mult,
        ib_outer_vv,
        ob_outer_vv,
        vv
    ]
)


Added materials to library from mat_lib.json
Added material ['plasma', 'flibe', 'uf4', 'tungsten', 'lead']


Specify path to cross section file of desired library.

In [5]:
#Provide path to cross section data
openmc.Materials.cross_sections = "/home/jlball/xs_data/endfb80_hdf5/cross_sections.xml"

## Geometry ##

The geometry is built in a separate file which uses the Paramak code to build a DAGMC model. Here it is imported and then a bounding region is provided.

In [6]:
dag_univ = openmc.DAGMCUniverse("arc2018.h5m")

# creates an edge of universe boundary surface
vac_surf = openmc.Sphere(r=10000, surface_id=9999, boundary_type="vacuum")

# adds reflective surface for the sector model at 0 degrees
reflective_1 = openmc.Plane(
    a=math.sin(0),
    b=-math.cos(0),
    c=0.0,
    d=0.0,
    surface_id=9991,
    boundary_type="reflective",
)

# specifies the region as below the universe boundary and inside the reflective surfaces
region = -vac_surf & -reflective_1 

# creates a cell from the region and fills the cell with the dagmc geometry
containing_cell = openmc.Cell(cell_id=9999, region=region, fill=dag_univ)

geometry = openmc.Geometry(root=[containing_cell])

## Settings ##

A simple line source geometry is defined.

In [7]:
# creates a simple isotropic neutron source in the center with 14MeV neutrons
source = openmc.Source()
# the distribution of radius is just a single value at the plasma major radius
radius = openmc.stats.Discrete([330.], [1])
# the distribution of source z values is just a single value
z_values = openmc.stats.Discrete([0], [1])
# the distribution of source azimuthal angles values is a uniform distribution between 0 and 0.5 Pi
# these angles must be the same as the reflective angles
angle = openmc.stats.Uniform(a=0., b=math.radians(180))
# this makes the ring source using the three distributions and a radius
source.space = openmc.stats.CylindricalIndependent(r=radius, phi=angle, z=z_values, origin=(0.0, 0.0, 0.0))
# sets the direction to isotropic
source.angle = openmc.stats.Isotropic()
# sets the energy distribution to a Muir distribution neutrons
source.energy = openmc.stats.Muir(e0=14080000.0, m_rat=5.0, kt=20000.0)

# specifies the simulation computational intensity
settings = openmc.Settings()
settings.batches = 10
settings.particles = 250000
settings.inactive = 0
settings.run_mode = "fixed source"
settings.source = source

## Tallies ##

Definition of tallies used.

In [8]:
# Tritium Breeding Tally:
trit_tally = openmc.Tally(name='tbr')
#trit_tally.filters = [blanket_filter]
trit_tally.scores = ["(n,Xt)"]

# Tritium Breeding Tally:
Be_tally = openmc.Tally(name='Be')
#Be_tally.filters = [blanket_filter]
Be_tally.scores = ["(n,2n)"]

# Plutonium Breeding Tally:
plut_tally = openmc.Tally(name='pbr')
plut_tally.nuclides = ['U238']
#plut_tally.filters = [blanket_filter]
plut_tally.scores = ["absorption"]

#Fission tally
fis_tally = openmc.Tally(name='fis')
#fis_tally.filters = [blanket_filter]
fis_tally.scores = ['fission']

tallies = openmc.Tallies([trit_tally, plut_tally, Be_tally, fis_tally])

## Run 

In [9]:
# builds the openmc model
model = openmc.Model(
    materials=materials, geometry=geometry, settings=settings, tallies=tallies
)

# starts the simulation
model.run(threads=12)

                                %%%%%%%%%%%%%%%
                           %%%%%%%%%%%%%%%%%%%%%%%%
                        %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                      %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                    %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                   %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                                    %%%%%%%%%%%%%%%%%%%%%%%%
                                     %%%%%%%%%%%%%%%%%%%%%%%%
                 ###############      %%%%%%%%%%%%%%%%%%%%%%%%
                ##################     %%%%%%%%%%%%%%%%%%%%%%%
                ###################     %%%%%%%%%%%%%%%%%%%%%%%
                ####################     %%%%%%%%%%%%%%%%%%%%%%
                #####################     %%%%%%%%%%%%%%%%%%%%%
                ######################     %%%%%%%%%%%%%%%%%%%%
                #######################     %%%%%%%%%%%%%%%%%%
                 #######################     %%%%%%%%%%%%%%%%%
                 #####################

PosixPath('/home/jlball/arc-proliferation/arc2018/statepoint.10.h5')